In [1]:
# # Step 1: Install Dependencies
# !pip install -q gdown inference-gpu
# !pip install -q onnxruntime-gpu==1.18.0 --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
# !pip install -q git+https://github.com/roboflow/sports.git
# !pip uninstall -y supervision && pip install -q supervision>=0.23.0


In [2]:
import cv2
import numpy as np
import torch
from tqdm import tqdm
import supervision as sv
from typing import Optional
from google.colab import userdata
from inference import get_model
from supervision import Detections, BoxAnnotator, EllipseAnnotator, TriangleAnnotator, LabelAnnotator
from typing import Generator, Iterable, List, TypeVar
import umap
from sklearn.cluster import KMeans
from transformers import AutoProcessor, SiglipVisionModel
from sports.common.team import TeamClassifier
from sports.configs.soccer import SoccerPitchConfiguration
from sports.common.view import ViewTransformer
from sports.annotators.soccer import (
    draw_pitch,
    draw_points_on_pitch,
    draw_pitch_voronoi_diagram
)
from google.colab import drive

ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support GroundingDINO model. Use pip install 'inference[grounding-dino]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration d

In [3]:
# Constants
SOURCE_VIDEO_PATH = "/kaggle/input/video-analysis/2.mp4"
OUTPUT_PITCH_VIDEO_PATH = "/kaggle/working/output_pitch_video.mp4"
OUTPUT_VORONOI_VIDEO_PATH = "/kaggle/working/output_voronoi_video.mp4"
OUTPUT_VORONOI_WITH__BLEND_VIDEO_PATH = "/kaggle/working/output_voronoi_with_blend_video.mp4"
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3
STRIDE = 50

In [4]:
# Initialize Models and configrations
CONFIG = SoccerPitchConfiguration()
ROBOFLOW_API_KEY = 'pQ2a96kNxTXy7kvwiXZe'
PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/11"
FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/14"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)
FIELD_DETECTION_MODEL = get_model(model_id=FIELD_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

# Initialize Tracker and Team Classifier
tracker = sv.ByteTrack()
team_classifier = TeamClassifier(device="cuda")

In [5]:
# Helper Function to Resolve Goalkeepers' Team ID
def resolve_goalkeepers_team_id(players: Detections, goalkeepers: Detections) -> np.ndarray:
    """
    Resolve the team ID of goalkeepers based on their proximity to players from each team.

    Args:
        players (Detections): Detections of players, classified by team.
        goalkeepers (Detections): Detections of goalkeepers whose team IDs need to be resolved.

    Returns:
        np.ndarray: An array of team IDs for the goalkeepers, where 0 and 1 represent the two teams.
    """

    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)
    goalkeepers_team_id = []
    for goalkeeper_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        goalkeepers_team_id.append(0 if dist_0 < dist_1 else 1)
    return np.array(goalkeepers_team_id)

In [6]:
def collect_and_train_team_classifier(
    source_video_path: str,
    player_detection_model,
    team_classifier,
    stride: int = STRIDE
) -> None:
    """
    Collect player crops from the video and train the team classifier.

    Args:
        source_video_path (str): Path to the source video.
        player_detection_model: Model for detecting players.
        team_classifier: Team classifier to be trained.
        stride (int): Stride for frame sampling. Defaults to STRIDE.
    """
    # Initialize frame generator
    frame_generator = sv.get_video_frames_generator(source_path=source_video_path, stride=stride)
    video_info = sv.VideoInfo.from_video_path(source_video_path)

    # Collect player crops for training the team classifier
    training_crops = []

    # First pass: Collect player crops for training
    for frame in tqdm(frame_generator, total=video_info.total_frames, desc='collecting crops'):
        result = player_detection_model.infer(frame, confidence=0.3)[0]
        detections = Detections.from_inference(result)
        players_detections = detections[detections.class_id == PLAYER_ID]
        player_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
        training_crops.extend(player_crops)

    # Train the team classifier
    print("Training Team Classifier...")
    team_classifier.fit(training_crops)
    print("Team Classifier training complete.")

In [7]:
def detect_and_classify_objects(frame, player_detection_model, team_classifier, tracker):
    """
    Detect and classify objects (ball, players, goalkeepers, referees) in a video frame.

    Args:
        frame: The input video frame (image) to process.
        player_detection_model: A pre-trained model used for detecting objects (e.g., ball, players, referees).
        team_classifier: A classifier used to assign players to their respective teams.
        tracker: A tracker used to track detected objects across frames.

    Returns:
        ball_detections: Detections of the ball in the frame.
        players_detections: Detections of players in the frame, classified by team.
        referees_detections: Detections of referees in the frame.
        all_detections: Merged detections of all objects (players, goalkeepers, referees).
    """
    result = player_detection_model.infer(frame, confidence=0.3)[0]
    detections = Detections.from_inference(result)
    detections = Detections.from_inference(result)


    # Separate Ball and Other Objects
    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)
    all_detections = detections[detections.class_id != BALL_ID]
    all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
    all_detections = tracker.update_with_detections(detections=all_detections)

    # Split Detections
    goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
    players_detections = all_detections[all_detections.class_id == PLAYER_ID]
    referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

    # Team Assignment
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    players_detections.class_id = team_classifier.predict(players_crops)
    goalkeepers_detections.class_id = resolve_goalkeepers_team_id(players_detections, goalkeepers_detections)
    referees_detections.class_id -= 1

    # Merge All Detections
    all_detections = Detections.merge([players_detections, goalkeepers_detections, referees_detections])
    labels = [f"#{tracker_id}" for tracker_id in all_detections.tracker_id]
    all_detections.class_id = all_detections.class_id.astype(int)

    return ball_detections, players_detections, referees_detections, all_detections

In [8]:
def detect_pitch_keypoints(frame, field_detection_model):
    """
    Detect pitch keypoints in a video frame using a field detection model.

    Args:
        frame: The input video frame (image) to process.
        field_detection_model: A pre-trained model used for detecting pitch keypoints.

    Returns:
        frame_reference_points: Coordinates of the detected keypoints in the video frame.
        pitch_reference_points: Corresponding coordinates of the keypoints on the standard pitch layout.
    """
    field_result = field_detection_model.infer(frame, confidence=0.1)[0]
    # print("Field Result Structure:", type(field_result), field_result)  # Debugging line

    # Check if predictions are empty
    if not field_result.predictions:
        print("No predictions detected in this frame.")
        return np.array([]), np.array([])

    key_points = sv.KeyPoints.from_inference(field_result)

    if key_points is None:
        print("No keypoints detected.")
        return np.array([]), np.array([])

    # Filter Keypoints by Confidence
    filter = key_points.confidence[0] > 0.5
    frame_reference_points = key_points.xy[0][filter]
    pitch_reference_points = np.array(CONFIG.vertices)[filter]

    return frame_reference_points, pitch_reference_points

In [9]:
def project_objects_to_pitch(transformer, ball_detections, players_detections, referees_detections):
    """
    Project the detected objects (ball, players, and referees) from the video frame to the pitch.

    Args:
        transformer: A ViewTransformer object that maps points from the video frame to the pitch.
        ball_detections: Detections of the ball in the video frame.
        players_detections: Detections of players in the video frame.
        referees_detections: Detections of referees in the video frame.

    Returns:
        pitch_ball_xy: Coordinates of the ball projected onto the pitch.
        pitch_players_xy: Coordinates of the players projected onto the pitch.
        pitch_referees_xy: Coordinates of the referees projected onto the pitch.
    """
    frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)

    players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_players_xy = transformer.transform_points(points=players_xy)

    referees_xy = referees_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_referees_xy = transformer.transform_points(points=referees_xy)

    return pitch_ball_xy, pitch_players_xy, pitch_referees_xy

In [10]:
def visualize_pitch_projection(config, pitch_ball_xy, pitch_players_xy, pitch_referees_xy, players_detections):
    """
    Visualize the pitch projection by drawing detected objects (ball, players, referees) on the pitch.

    Args:
        config: Configuration object containing settings for visualization.
        pitch_ball_xy: Coordinates of the ball projected onto the pitch.
        pitch_players_xy: Coordinates of the players projected onto the pitch.
        pitch_referees_xy: Coordinates of the referees projected onto the pitch.
        players_detections: Detections of players, including their classification into teams.

    Returns:
        annotated_pitch_frame: Annotated pitch frame with visualized projections of the ball, players, and referees.
    """
    annotated_pitch_frame = draw_pitch(config)
    annotated_pitch_frame = draw_points_on_pitch(
        config=config,
        xy=pitch_ball_xy,
        face_color=sv.Color.WHITE,
        edge_color=sv.Color.BLACK,
        radius=10,
        pitch=annotated_pitch_frame
    )
    annotated_pitch_frame = draw_points_on_pitch(
        config=config,
        xy=pitch_players_xy[players_detections.class_id == 0],
        face_color=sv.Color.from_hex('00BFFF'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=annotated_pitch_frame
    )
    annotated_pitch_frame = draw_points_on_pitch(
        config=config,
        xy=pitch_players_xy[players_detections.class_id == 1],
        face_color=sv.Color.from_hex('FF1493'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=annotated_pitch_frame
    )
    annotated_pitch_frame = draw_points_on_pitch(
        config=config,
        xy=pitch_referees_xy,
        face_color=sv.Color.from_hex('FFD700'),
        edge_color=sv.Color.BLACK,
        radius=16,
        pitch=annotated_pitch_frame
    )
    return annotated_pitch_frame

In [11]:
def draw_pitch_voronoi_diagram(
    config: SoccerPitchConfiguration,
    team_1_xy: np.ndarray,
    team_2_xy: np.ndarray,
    team_1_color: sv.Color = sv.Color.RED,
    team_2_color: sv.Color = sv.Color.WHITE,
    opacity: float = 0.5,
    padding: int = 50,
    scale: float = 0.1,
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Draws a Voronoi diagram on a soccer pitch representing the control areas of two
    teams with smooth color transitions.

    Args:
        config (SoccerPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        team_1_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 1.
        team_2_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 2.
        team_1_color (sv.Color, optional): Color representing the control area of
            team 1. Defaults to sv.Color.RED.
        team_2_color (sv.Color, optional): Color representing the control area of
            team 2. Defaults to sv.Color.WHITE.
        opacity (float, optional): Opacity of the Voronoi diagram overlay.
            Defaults to 0.5.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw the
            Voronoi diagram on. If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the soccer pitch with the Voronoi diagram overlay.
    """
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    scaled_width = int(config.width * scale)
    scaled_length = int(config.length * scale)

    voronoi = np.zeros_like(pitch, dtype=np.uint8)

    team_1_color_bgr = np.array(team_1_color.as_bgr(), dtype=np.uint8)
    team_2_color_bgr = np.array(team_2_color.as_bgr(), dtype=np.uint8)

    y_coordinates, x_coordinates = np.indices((
        scaled_width + 2 * padding,
        scaled_length + 2 * padding
    ))

    y_coordinates -= padding
    x_coordinates -= padding

    def calculate_distances(xy, x_coordinates, y_coordinates):
        return np.sqrt((xy[:, 0][:, None, None] * scale - x_coordinates) ** 2 +
                       (xy[:, 1][:, None, None] * scale - y_coordinates) ** 2)

    distances_team_1 = calculate_distances(team_1_xy, x_coordinates, y_coordinates)
    distances_team_2 = calculate_distances(team_2_xy, x_coordinates, y_coordinates)

    min_distances_team_1 = np.min(distances_team_1, axis=0)
    min_distances_team_2 = np.min(distances_team_2, axis=0)

    # Increase steepness of the blend effect
    steepness = 15  # Increased steepness for sharper transition
    distance_ratio = min_distances_team_2 / np.clip(min_distances_team_1 + min_distances_team_2, a_min=1e-5, a_max=None)
    blend_factor = np.tanh((distance_ratio - 0.5) * steepness) * 0.5 + 0.5

    # Create the smooth color transition
    for c in range(3):  # Iterate over the B, G, R channels
        voronoi[:, :, c] = (blend_factor * team_1_color_bgr[c] +
                            (1 - blend_factor) * team_2_color_bgr[c]).astype(np.uint8)

    overlay = cv2.addWeighted(voronoi, opacity, pitch, 1 - opacity, 0)

    return overlay

In [12]:
def visualize_voronoi_diagram(
    config: SoccerPitchConfiguration,
    team_1_xy: np.ndarray,
    team_2_xy: np.ndarray,
    team_1_color: sv.Color = sv.Color.from_hex('00BFFF'),
    team_2_color: sv.Color = sv.Color.from_hex('FF1493'),
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Visualize the Voronoi diagram for two teams on the pitch.

    Args:
        config (SoccerPitchConfiguration): Configuration object for the pitch.
        team_1_xy (np.ndarray): Coordinates of team 1 players.
        team_2_xy (np.ndarray): Coordinates of team 2 players.
        team_1_color (sv.Color): Color for team 1.
        team_2_color (sv.Color): Color for team 2.
        pitch (Optional[np.ndarray]): Existing pitch image. If None, a new pitch is created.

    Returns:
        np.ndarray: Image with the Voronoi diagram.
    """
    if pitch is None:
        pitch = draw_pitch(config)

    annotated_frame = draw_pitch_voronoi_diagram(
        config=config,
        team_1_xy=team_1_xy,
        team_2_xy=team_2_xy,
        team_1_color=team_1_color,
        team_2_color=team_2_color,
        pitch=pitch
    )

    return annotated_frame

In [13]:
def visualize_voronoi_diagram_with_blend(
    config: SoccerPitchConfiguration,
    team_1_xy: np.ndarray,
    team_2_xy: np.ndarray,
    ball_xy: np.ndarray,
    team_1_color: sv.Color = sv.Color.from_hex('00BFFF'),
    team_2_color: sv.Color = sv.Color.from_hex('FF1493'),
    ball_color: sv.Color = sv.Color.WHITE,
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Visualize the Voronoi diagram with smooth color transitions and overlay ball/player positions.

    Args:
        config (SoccerPitchConfiguration): Configuration object for the pitch.
        team_1_xy (np.ndarray): Coordinates of team 1 players.
        team_2_xy (np.ndarray): Coordinates of team 2 players.
        ball_xy (np.ndarray): Coordinates of the ball.
        team_1_color (sv.Color): Color for team 1.
        team_2_color (sv.Color): Color for team 2.
        ball_color (sv.Color): Color for the ball.
        pitch (Optional[np.ndarray]): Existing pitch image. If None, a new pitch is created.

    Returns:
        np.ndarray: Image with the Voronoi diagram and player/ball positions.
    """
    if pitch is None:
        pitch = draw_pitch(config, background_color=sv.Color.WHITE, line_color=sv.Color.BLACK)

    # Draw Voronoi diagram with blend
    annotated_frame = draw_pitch_voronoi_diagram(
        config=config,
        team_1_xy=team_1_xy,
        team_2_xy=team_2_xy,
        team_1_color=team_1_color,
        team_2_color=team_2_color,
        pitch=pitch
    )

    # Draw ball and players
    annotated_frame = draw_points_on_pitch(
        config=config,
        xy=ball_xy,
        face_color=ball_color,
        edge_color=ball_color,
        radius=8,
        thickness=1,
        pitch=annotated_frame
    )
    annotated_frame = draw_points_on_pitch(
        config=config,
        xy=team_1_xy,
        face_color=team_1_color,
        edge_color=sv.Color.WHITE,
        radius=16,
        thickness=1,
        pitch=annotated_frame
    )
    annotated_frame = draw_points_on_pitch(
        config=config,
        xy=team_2_xy,
        face_color=team_2_color,
        edge_color=sv.Color.WHITE,
        radius=16,
        thickness=1,
        pitch=annotated_frame
    )

    return annotated_frame

In [14]:
def process_video_for_pitch_projection(
    source_video_path: str,
    output_pitch_video_path: str,
    output_voronoi_video_path: str,
    output_voronoi_with_blend_video_path: str,
    player_detection_model,
    field_detection_model,
    team_classifier,
    tracker,
    config
):
    """
    Process a video to generate pitch projections, Voronoi diagrams, and blended Voronoi visualizations.

    Args:
        source_video_path (str): Path to the input video file.
        output_pitch_video_path (str): Path to save the pitch projection video.
        output_voronoi_video_path (str): Path to save the Voronoi diagram video.
        output_voronoi_with_blend_video_path (str): Path to save the blended Voronoi diagram video.
        player_detection_model: Model used for detecting players in the video frames.
        field_detection_model: Model used for detecting the soccer field keypoints in the video frames.
        team_classifier: Model used for classifying players into teams.
        tracker: Tracker used for tracking detected objects across frames.
        config: Configuration object containing settings for visualization and processing.
    """

    # Train the team classifier
    collect_and_train_team_classifier(source_video_path, player_detection_model, team_classifier)

    # Initialize video writer
    video_info = sv.VideoInfo.from_video_path(source_video_path)

    writer_pitch = cv2.VideoWriter(output_pitch_video_path, cv2.VideoWriter_fourcc(*"mp4v"), video_info.fps, (video_info.width, video_info.height))
    writer_voronoi = cv2.VideoWriter(output_voronoi_video_path, cv2.VideoWriter_fourcc(*"mp4v"), video_info.fps, (video_info.width, video_info.height))
    writer_voronoi_with_blend = cv2.VideoWriter(output_voronoi_with_blend_video_path, cv2.VideoWriter_fourcc(*"mp4v"), video_info.fps, (video_info.width, video_info.height))

    if not writer_pitch.isOpened() or  writer_voronoi.isOpened() or writer_voronoi_with_blend.isOpened() :
        print("Error: Video writer could not be initialized.")

    # Process each frame
    frame_generator = sv.get_video_frames_generator(source_video_path)


    for frame in tqdm(frame_generator, total=video_info.total_frames):
        # Detect and classify objects
        ball_detections, players_detections, referees_detections, all_detections = detect_and_classify_objects(
            frame, player_detection_model, team_classifier, tracker
        )
        # Skip frame if no objects are detected
        if len(all_detections) == 0:
            print("No objects detected in the frame.")
            continue

        # Detect pitch keypoints
        frame_reference_points, pitch_reference_points = detect_pitch_keypoints(frame, field_detection_model)

        if len(frame_reference_points) == 0 or len(pitch_reference_points) == 0:
            print("No pitch keypoints detected in the frame.")
            continue

        # Initialize View Transformer
        transformer = ViewTransformer(source=frame_reference_points, target=pitch_reference_points)

        # Project objects to pitch
        pitch_ball_xy, pitch_players_xy, pitch_referees_xy = project_objects_to_pitch(
            transformer, ball_detections, players_detections, referees_detections
        )

        # Visualize pitch projection
        annotated_pitch_frame = visualize_pitch_projection(
            config, pitch_ball_xy, pitch_players_xy, pitch_referees_xy, players_detections
        )
        annotated_pitch_frame = cv2.resize(annotated_pitch_frame, (video_info.width, video_info.height))

        annotated_voronoi_frame = visualize_voronoi_diagram(
            config = config,
            team_1_xy=pitch_players_xy[players_detections.class_id == 0],
            team_2_xy=pitch_players_xy[players_detections.class_id == 1],
            pitch=draw_pitch(config)
            )
        annotated_voronoi_frame = cv2.resize(annotated_voronoi_frame, (video_info.width, video_info.height))

        # Visualize Voronoi diagram with blend
        annotated_voronoi_frame_with_blend = visualize_voronoi_diagram_with_blend(
            config=config,
            team_1_xy=pitch_players_xy[players_detections.class_id == 0],
            team_2_xy=pitch_players_xy[players_detections.class_id == 1],
            ball_xy=pitch_ball_xy,
            pitch=draw_pitch(config)
        )
        annotated_voronoi_frame_with_blend = cv2.resize(annotated_voronoi_frame_with_blend, (video_info.width, video_info.height))

        # Write pitch frame to output video
        writer_pitch.write(annotated_pitch_frame)
        writer_voronoi.write(annotated_voronoi_frame)
        writer_voronoi_with_blend.write(annotated_voronoi_frame_with_blend)

    # Release Video Writer
    writer_pitch.release()
    writer_voronoi.release()
    writer_voronoi_with_blend.release()

    print("Processing Complete. Output saved at:", output_pitch_video_path)
    print("Processing Complete. Output saved at:", output_voronoi_video_path)
    print("Processing Complete. Output saved at:", output_voronoi_with_blend_video_path)

In [15]:
process_video_for_pitch_projection(
        SOURCE_VIDEO_PATH,
        OUTPUT_PITCH_VIDEO_PATH,
        OUTPUT_VORONOI_VIDEO_PATH,
        OUTPUT_VORONOI_WITH__BLEND_VIDEO_PATH,
        PLAYER_DETECTION_MODEL,
        FIELD_DETECTION_MODEL,
        team_classifier,
        tracker,
        CONFIG
    )

collecting crops:   2%|▏         | 28/1399 [00:06<04:54,  4.66it/s]


Training Team Classifier...


Embedding extraction: 14it [00:03,  3.74it/s]


Team Classifier training complete.


  0%|          | 0/1399 [00:00<?, ?it/s]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  6.10it/s]
  0%|          | 1/1399 [00:03<1:32:54,  3.99s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  6.49it/s]
  0%|          | 2/1399 [00:04<43:33,  1.87s/it]  
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  7.05it/s]
  0%|          | 3/1399 [00:04<27:35,  1.19s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  7.08it/s]
  0%|          | 4/1399 [00:05<20:05,  1.16it/s]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  6.79it/s]
  0%|          | 5/1399 [00:05<16:01,  1.45it/s]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  6.89it/s]
  0%|          | 6/1399 [00:05<13:32,  1.71it/s]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  6.93it/s]
  1%|          | 7/1399 [00:06<11:57,  1.94it/s]
Embedding 

...

No objects detected in frame 1356.



Embedding extraction: 1it [00:00, 60.73it/s]
 97%|█████████▋| 1359/1399 [08:02<00:07,  5.19it/s]

No objects detected in frame 1358.



Embedding extraction: 1it [00:00, 59.18it/s]
 97%|█████████▋| 1360/1399 [08:02<00:07,  4.90it/s]
Embedding extraction: 1it [00:00, 60.73it/s]
 97%|█████████▋| 1361/1399 [08:03<00:08,  4.70it/s]
Embedding extraction: 1it [00:00, 11.93it/s]
 97%|█████████▋| 1362/1399 [08:03<00:08,  4.14it/s]
Embedding extraction: 1it [00:00, 21.50it/s]
 97%|█████████▋| 1363/1399 [08:03<00:08,  4.02it/s]
Embedding extraction: 1it [00:00, 21.41it/s]
 97%|█████████▋| 1364/1399 [08:03<00:08,  3.91it/s]
Embedding extraction: 1it [00:00, 18.31it/s]
 98%|█████████▊| 1365/1399 [08:04<00:08,  3.82it/s]
Embedding extraction: 1it [00:00, 14.27it/s]
 98%|█████████▊| 1366/1399 [08:04<00:08,  3.69it/s]
Embedding extraction: 1it [00:00, 13.67it/s]
 98%|█████████▊| 1367/1399 [08:04<00:08,  3.58it/s]
Embedding extraction: 1it [00:00, 10.16it/s]
 98%|█████████▊| 1368/1399 [08:05<00:09,  3.42it/s]
Embedding extraction: 1it [00:00, 10.43it/s]
 98%|█████████▊| 1369/1399 [08:05<00:09,  3.32it/s]
Embedding extraction: 1it [00

Processing Complete. Output saved at: /kaggle/working/output_pitch_video.mp4
Processing Complete. Output saved at: /kaggle/working/output_voronoi_video.mp4
Processing Complete. Output saved at: /kaggle/working/output_voronoi_with_blend_video.mp4
